In [1]:
import pandas as pd
import pickle
import copy
import cobra
import numpy as np
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model,write_sbml_model
from cobra import Model, Reaction, Metabolite
from functools import reduce 
%matplotlib inline

In [2]:
mutant1 = read_sbml_model('mutant1.xml')

In [3]:
mutant1.medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


{'EX_ca2_e': 1000.0,
 'EX_cit_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_hdca_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_met__L_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [4]:
model2 = read_sbml_model('model2.xml')

In [5]:
model2.medium

{'EX_ca2_e': 1000.0,
 'EX_cit_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_met__L_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_hdca_e': 1000.0}

In [6]:
mutant1_reactions = {}
for i in mutant1.reactions:
    x=i.id
    y=i.build_reaction_string()
    mutant1_reactions[x] = y

In [7]:
model2_reactions = {}
for i in model2.reactions:
    x=i.id
    y=i.build_reaction_string()
    model2_reactions[x] = y

In [8]:
model2_unique = {}
for x in model2_reactions:
    if x in mutant1_reactions: continue
    else:
        y = model2_reactions[x]
        if x == 'BIOMASS_SCO': continue
        else:
            model2_unique[x] = y

In [9]:
model2.slim_optimize()

14.657320838057537

### Add sink reaction for acetyl coA

In [10]:
print("exchanges", len(mutant1.exchanges))
print("demands", len(mutant1.demands))
print("sinks", len(mutant1.sinks))

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


exchanges 343
demands 0
sinks 6


In [11]:
print("exchanges", len(model2.exchanges))
print("demands", len(model2.demands))
print("sinks", len(model2.sinks))

exchanges 158
demands 4
sinks 0


In [12]:
print("sinks", mutant1.sinks)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


sinks [<Reaction sink_2ohph_c at 0x1833b070670>, <Reaction sink_4hba_c at 0x1833b0548e0>, <Reaction sink_amob_c at 0x1833b070a30>, <Reaction sink_hemeO_c at 0x1833b070c10>, <Reaction sink_lipopb_c at 0x1833b070c70>, <Reaction sink_sheme_c at 0x1833b07f7f0>]


In [13]:
mutant1.reactions.get_by_id('sink_2ohph_c')

Reaction identifier,sink_2ohph_c
Name,R_sink_2ohph_c
Memory address,0x1833b070670
Stoichiometry,2ohph_c --> 2-Octaprenyl-6-hydroxyphenol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
reaction = Reaction('sink_accoa_c')
reaction.name = 'R_sink_accoa_c'
reaction.subsystem = 'Metabolite'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.gene_reaction_rule = ''

In [15]:
reaction.add_metabolites({mutant1.metabolites.accoa_c:-1})

In [16]:
mutant1.add_reactions([reaction])

In [17]:
mutant1.reactions.get_by_id('sink_accoa_c')

Reaction identifier,sink_accoa_c
Name,R_sink_accoa_c
Memory address,0x1833b684e50
Stoichiometry,accoa_c --> Acetyl-CoA -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Set sink reaction as the objective

In [18]:
mutant1.objective='sink_accoa_c'

In [19]:
c = cobra.flux_analysis.pfba(mutant1).to_frame()

In [20]:
c.to_excel('Acetyl_coa_sink_fluxes.xlsx')

In [21]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent  Flux Reaction                                         Definition
 80.65% 108.2      PDH coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
 19.35% 25.97      PFL                  coa_c + pyr_c --> accoa_c + for_c

Consuming Reactions
-------------------
Percent   Flux     Reaction                                      Definition
 90.32% -121.2           CS accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
  9.68% -12.99 sink_accoa_c                                    accoa_c --> 


### Adding Glucose to medium

In [22]:
medium = mutant1.medium
medium["EX_cit_e"] = 0.0
medium['EX_glc__D_e'] = 1000
mutant1.medium = medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [23]:
solution = cobra.flux_analysis.pfba(mutant1)

### Find out why cells cannot use lipids. if you only use lipids can you produce acetyl coA.

In [24]:
mutant1.objective='ATPM'

In [25]:
solution = cobra.flux_analysis.pfba(mutant1)

In [26]:
x = solution.to_frame()

In [27]:
x.to_excel('T.xlsx')

In [28]:
solution1 = cobra.flux_analysis.pfba(model2)

In [29]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent Flux Reaction                                                     Definition
100.00%  500    ACTD2 actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c

Consuming Reactions
-------------------
Percent Flux Reaction                           Definition
100.00% -500    GLYAT accoa_c + gly_c <=> 2aobut_c + coa_c


In [30]:
print(mutant1.metabolites.actn__R_c.summary())

actn__R_c
Formula: C4H8O2

Producing Reactions
-------------------
Percent Flux Reaction                            Definition
100.00%  500    ACLDC alac__S_c + h_c --> actn__R_c + co2_c

Consuming Reactions
-------------------
Percent Flux Reaction                                                     Definition
100.00% -500    ACTD2 actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c


In [31]:
print(mutant1.metabolites.alac__S_c.summary())

alac__S_c
Formula: C5H7O4

Producing Reactions
-------------------
Percent Flux Reaction                            Definition
100.00%  500     ACLS h_c + 2.0 pyr_c --> alac__S_c + co2_c

Consuming Reactions
-------------------
Percent Flux Reaction                            Definition
100.00% -500    ACLDC alac__S_c + h_c --> actn__R_c + co2_c


In [67]:
solution['ATPM']

999.9999999999999

In [66]:
print(mutant1.metabolites.atp_c.summary())

atp_c
=====
Formula: C10H12N5O13P3

Producing Reactions
-------------------
Percent Flux Reaction                            Definition
 66.67% 1000      PGK     3pg_c + atp_c <=> 13dpg_c + adp_c
 33.33%  500      PYK adp_c + h_c + pep_c --> atp_c + pyr_c

Consuming Reactions
-------------------
Percent  Flux Reaction                            Definition
 66.67% -1000     ATPM  atp_c + h2o_c --> adp_c + h_c + pi_c
 33.33%  -500      PFK atp_c + f6p_c --> adp_c + fdp_c + h_c


In [32]:
mutant1.reactions.get_by_id("PDH").build_reaction_string(use_metabolite_names = True)

'Coenzyme A + Nicotinamide adenine dinucleotide + Pyruvate --> Acetyl-CoA + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced'

In [33]:
mutant1.reactions.get_by_id("PFL").build_reaction_string(use_metabolite_names = True)

'Coenzyme A + Pyruvate --> Acetyl-CoA + Formate'

In [34]:
mutant1.reactions.get_by_id("ACTD2").build_reaction_string(use_metabolite_names = True)

' R  Acetoin C4H8O2 + Coenzyme A + Nicotinamide adenine dinucleotide --> Acetaldehyde + Acetyl-CoA + H+ + Nicotinamide adenine dinucleotide - reduced'

In [35]:
print(model2.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent    Flux Reaction                                Definition
 16.62%   44.48  ACACT2r    accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
 16.62%   44.48  ACACT3r    accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
 16.68%   44.64  ACACT4r    accoa_c + occoa_c <=> 3odcoa_c + coa_c
 16.68%   44.64  ACACT5r  accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
 16.68%   44.64  ACACT6r accoa_c + ddcacoa_c <=> 3otdcoa_c + coa_c
 16.68%   44.64  ACACT7r   accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c
  0.03% 0.08019     PTAr         accoa_c + pi_c <=> actp_c + coa_c

Consuming Reactions
-------------------
Percent     Flux Reaction                                                 Definition
 91.29%   -244.3  ACACT1r                            2.0 accoa_c <=> aacoa_c + coa_c
  5.42%   -14.49   ACCOAC accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c
  1.35%   -3.624    CITMS              accoa_c + h2o_c + pyr_c <=> citm__R_c + coa_c
  1.91%   

In [36]:
 mutant1.metabolites.get_by_id('aacoa_c').summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [37]:
 mutant1.metabolites.get_by_id('coa_c').summary()

Percent,Flux,Reaction,Definition
100.00%,500,GLYAT,accoa_c + gly_c <=> 2aobut_c + coa_c
Percent,Flux,Reaction,Definition
100.00%,-500,ACTD2,actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c


In [38]:
model2.reactions.get_by_id("ACACT2r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Butanoyl-CoA <=> 3-Oxohexanoyl-CoA + Coenzyme A'

In [39]:
model2.reactions.get_by_id("ACACT3r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Hexanoyl-CoA (n-C6:0CoA) <=> 3-Oxooctanoyl-CoA + Coenzyme A'

In [40]:
model2.reactions.get_by_id("ACACT4r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Octanoyl-CoA (n-C8:0CoA) <=> 3-Oxodecanoyl-CoA + Coenzyme A'

In [41]:
model2.reactions.get_by_id("ACACT5r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Decanoyl-CoA (n-C10:0CoA) <=> 3-Oxododecanoyl-CoA + Coenzyme A'

In [42]:
model2.reactions.get_by_id("ACACT6r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Dodecanoyl-CoA (n-C12:0CoA) <=> 3-Oxotetradecanoyl-CoA + Coenzyme A'

In [43]:
model2.reactions.get_by_id("ACACT7r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Tetradecanoyl-CoA (n-C14:0CoA) <=> 3-Oxohexadecanoyl-CoA + Coenzyme A'

In [44]:
model2.reactions.get_by_id("PTAr").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Phosphate <=> Acetyl phosphate + Coenzyme A'

In [45]:
model2.reactions.get_by_id("ACACT1r").build_reaction_string(use_metabolite_names = True)

'2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A'

In [46]:
model2.reactions.get_by_id("ACCOAC").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + ATP + Bicarbonate --> ADP + H+ + Malonyl-CoA + Phosphate'

In [47]:
model2.reactions.get_by_id("CITMS").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + H2O + Pyruvate <=> (R)-2-Methylmalate + Coenzyme A'

In [48]:
model2.reactions.get_by_id("GLYAT").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Glycine <=> L-2-Amino-3-oxobutanoate + Coenzyme A'

In [49]:
mutant1.reactions.get_by_id("ACTD2").build_reaction_string(use_metabolite_names = True)

' R  Acetoin C4H8O2 + Coenzyme A + Nicotinamide adenine dinucleotide --> Acetaldehyde + Acetyl-CoA + H+ + Nicotinamide adenine dinucleotide - reduced'

In [50]:
for x in mutant1.metabolites.get_by_id('2aobut_c').reactions:
    print(x.id)
    print(x.build_reaction_string(use_metabolite_names = False))
    print(solution[x.id])

ATHRDHr
athr__L_c + nadp_c <=> 2aobut_c + h_c + nadph_c
-500.0
GLYAT
accoa_c + gly_c <=> 2aobut_c + coa_c
500.0
THRD
nad_c + thr__L_c --> 2aobut_c + h_c + nadh_c
0.0


In [51]:
li = ['3ohcoa_c','3oocoa_c','3odcoa_c','3oddcoa_c','3otdcoa_c','3ohdcoa_c']
for a in li:
    print(a)
    print(mutant1.metabolites.get_by_id(a).summary())

3ohcoa_c
3ohcoa_c
Formula: C27H40N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
3oocoa_c
3oocoa_c
Formula: C29H44N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
3odcoa_c
3odcoa_c
Formula: C31H48N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
3oddcoa_c
3oddcoa_c
Formula: C33H52N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-----------

In [52]:
li = ['3hhcoa_c','3ohcoa_c','3oocoa_c','3odcoa_c','3oddcoa_c','3otdcoa_c','3ohdcoa_c','3ohcoa_c','3hhdcoa_c','btcoa_c','hx2coa_c']
for a in li:
    print(a)
    for x in mutant1.metabolites.get_by_id(a).reactions:
        print(x.id)
        print(x.build_reaction_string(use_metabolite_names = False))
        print(solution[x.id])

3hhcoa_c
HACD2i
3hhcoa_c + nad_c --> 3ohcoa_c + h_c + nadh_c
0.0
ECOAH2
3hhcoa_c <=> h2o_c + hx2coa_c
1.2713048220173763e-14
HACD2
3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c
1.2713048220173763e-14
3ohcoa_c
ACACT11
3ohcoa_c + coa_c --> accoa_c + btcoa_c
0.0
HACD2i
3hhcoa_c + nad_c --> 3ohcoa_c + h_c + nadh_c
0.0
HACD2
3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c
1.2713048220173763e-14
ACACT2r
accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
1.2713048220173764e-14
3oocoa_c
HACD3i
3hocoa_c + nad_c --> 3oocoa_c + h_c + nadh_c
0.0
ACACT3r
accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
1.271304822017376e-14
ACACT8
3oocoa_c + coa_c --> accoa_c + hxcoa_c
0.0
HACD3
3oocoa_c + h_c + nadh_c <=> 3hocoa_c + nad_c
1.2713048220173757e-14
3odcoa_c
HACD4i
3hdcoa_c + nad_c --> 3odcoa_c + h_c + nadh_c
0.0
ACACT13
3odcoa_c + coa_c --> accoa_c + occoa_c
0.0
HACD4
3odcoa_c + h_c + nadh_c <=> 3hdcoa_c + nad_c
1.2713048220173757e-14
ACACT4r
accoa_c + occoa_c <=> 3odcoa_c + coa_c
1.2713048220173757e-14
3oddcoa_c
HACD5
3oddcoa

In [53]:
li = ['3hhcoa_c','3ohcoa_c','3oocoa_c','3odcoa_c','3oddcoa_c','3otdcoa_c','3ohdcoa_c','3ohcoa_c','3hhdcoa_c','btcoa_c']
for a in li:
    print(a)
    print(model2.metabolites.get_by_id(a).summary())

3hhcoa_c
3hhcoa_c
Formula: C27H42N7O18P3S

Producing Reactions
-------------------
Percent  Flux Reaction                    Definition
100.00% 44.48   ECOAH2 3hhcoa_c <=> h2o_c + hx2coa_c

Consuming Reactions
-------------------
Percent   Flux Reaction                                   Definition
100.00% -44.48    HACD2 3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c
3ohcoa_c
3ohcoa_c
Formula: C27H40N7O18P3S

Producing Reactions
-------------------
Percent  Flux Reaction                                   Definition
100.00% 44.48    HACD2 3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c

Consuming Reactions
-------------------
Percent   Flux Reaction                             Definition
100.00% -44.48  ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
3oocoa_c
3oocoa_c
Formula: C29H44N7O18P3S

Producing Reactions
-------------------
Percent  Flux Reaction                                   Definition
100.00% 44.48    HACD3 3oocoa_c + h_c + nadh_c <=> 3hocoa_c + nad_c

Consuming Reactions
------

In [54]:
reactions_to_add = {}
ri = ['CITMS','BCOATA', 'hdca_Transp_c0']
for x in model2_reactions:
    if x in ri:
        y = model2_reactions[x]
        reactions_to_add[x] = y

In [55]:
for x in reactions_to_add:
    reaction = model2.reactions.get_by_id(x)
    print(reaction)
    mutant1.add_reactions([reaction])
    print(mutant1.slim_optimize())

CITMS: accoa_c + h2o_c + pyr_c <=> citm__R_c + coa_c
1000.0
BCOATA: ACP_c + btcoa_c <=> butACP_c + coa_c
1000.0
hdca_Transp_c0: hdca_e <=> hdca_c
1000.0


In [56]:
model2.reactions.get_by_id("CITMS").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + H2O H2O + Pyruvate <=> (R)-2-Methylmalate + Coenzyme A'

In [57]:
model2.reactions.get_by_id("BCOATA").build_reaction_string(use_metabolite_names = True)

'Acyl carrier protein + Butanoyl-CoA <=> Butyryl-ACP (n-C4:0ACP) + Coenzyme A'

https://www.genome.jp/pathway/rn01212+R04429

In [58]:
mutant1.slim_optimize()

1000.0

In [59]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent Flux Reaction                                                     Definition
100.00%  500    ACTD2 actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c

Consuming Reactions
-------------------
Percent  Flux Reaction                                Definition
 25.00%  -125  ACACT1r           2.0 accoa_c <=> aacoa_c + coa_c
 12.50% -62.5  ACACT2r    accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
 12.50% -62.5  ACACT3r    accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
 12.50% -62.5  ACACT4r    accoa_c + occoa_c <=> 3odcoa_c + coa_c
 12.50% -62.5  ACACT5r  accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
 12.50% -62.5  ACACT6r accoa_c + ddcacoa_c <=> 3otdcoa_c + coa_c
 12.50% -62.5  ACACT7r   accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c


In [60]:
mutant1.objective='ATPM'

In [61]:
mutant1.slim_optimize()

1000.0000000000007

In [62]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent Flux Reaction                                                     Definition
100.00%  500    ACTD2 actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c

Consuming Reactions
-------------------
Percent  Flux Reaction                                Definition
 25.00%  -125  ACACT1r           2.0 accoa_c <=> aacoa_c + coa_c
 12.50% -62.5  ACACT2r    accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
 12.50% -62.5  ACACT3r    accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
 12.50% -62.5  ACACT4r    accoa_c + occoa_c <=> 3odcoa_c + coa_c
 12.50% -62.5  ACACT5r  accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
 12.50% -62.5  ACACT6r accoa_c + ddcacoa_c <=> 3otdcoa_c + coa_c
 12.50% -62.5  ACACT7r   accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c


In [63]:
model2.objective='ATPM'

In [64]:
model2.slim_optimize()

1000.0

In [65]:
#print(model2.metabolites.accoa_c.summary())